In [1]:
import sys
sys.path.insert(0, '../../')
from questions_construction.main import *

In [2]:
question_multiplicity = 1
upper_instance = 11
all_questions_tmp = []
all_questions_rand_tmp = []
for domain_class in ALL_DOMAIN_CLASSES:
    domain = domain_class(is_random_sub=False, is_ramifications=False) # for questions, is_ramifications does not matter T/F, only for prompts
    for i in range(1, upper_instance):
        instance_name = f'Instance_{i}'
        jsonl_instance = open_jsonl(STATES_ACTIONS_PATH + f'/{domain.DOMAIN_NAME}/{instance_name}.jsonl')

        all_questions = EffectsQuestions(jsonl_instance, domain, instance_name).create_questions(question_multiplicity, PLAN_LENGTHS)
        all_questions_tmp += all_questions

        random_sub_all_questions = deepcopy(all_questions)
        for d in random_sub_all_questions:
            for k in [OUT_OBJ_ANSWER, OUT_OBJ_QUESTION, OUT_OBJ_INITIAL_STATE_NL]:
                d[k] = domain.to_random_substring(d[k])
            d['with_random_sub'] = True
        all_questions_rand_tmp += random_sub_all_questions

In [ ]:
true_e = 0
true_s = 0
false_e = 0
false_s = 0
for d in all_questions_tmp:
    if 'iter_1' in d['question_name']:
        if 'change' in d['question'][-20:] and d[OUT_OBJ_ANSWER] == 'True':
            true_e += 1
        elif 'change' not in d['question'][-20:] and d[OUT_OBJ_ANSWER] == 'True':
            true_s += 1
        elif 'change' in d['question'][-20:] and d[OUT_OBJ_ANSWER] == 'False':
            false_e += 1
        elif 'change' not in d['question'][-20:] and d[OUT_OBJ_ANSWER] == 'False':
            false_s += 1
print(true_e, true_s, false_e, false_s)

In [6]:
sample = random.sample(all_questions_tmp, 100)
for d in sample:
    if 'change' in d['question'][-20:]:
        print(d['question_name'])
        print(d['question'])
        print(d[OUT_OBJ_ANSWER])
        print()

iter_1_question_7
Given the initial condition, the following actions are performed: instrument3 on satellite1 is switched on, on satellite0, instrument0 is switched on, satellite1 turns from groundstation4 to star6, instrument3 is calibrated on satellite1 to star6, satellite1 turns to planet14 from star6, satellite1's instrument3 takes an image of planet14 in spectrograph1, from planet14, satellite1 turns to star10, image of star10 is taken with instrument3 on satellite1 in spectrograph1, satellite1 turns to star12 from star10, image of star12 is taken with instrument3 on satellite1 in spectrograph1, satellite1 turns to star0 from star12, satellite0 turns to groundstation2 from star1, instrument0 is calibrated on satellite0 to groundstation2, from groundstation2, satellite0 turns to phenomenon15 and image of phenomenon15 is taken with instrument0 on satellite0 in spectrograph0 to reach the current state. In this state, if satellite0 turns to star11 from phenomenon15, is it True or Fals

In [4]:
save_jsonl(all_questions_tmp, f'effects_tf.jsonl')
save_jsonl(all_questions_rand_tmp, f'effects_tf.random_sub.jsonl')